# SQL Querying

This notebook can be used to query tables in the Congressional Data database. In order to use this notebook, you will need to set an environment variable 'CD_DWH' to the database connection string. If you do not have the credentials, please slack us at #datasci-congressdata channel and/or talk to a project lead.

**It is best practice to not hard code database URI strings directly in notebooks or code as when we push to Github, that would mean credentials are public for anyone to see.**

In [1]:
import os
import sys

import pandas as pd
pd.options.display.max_columns = 999
import sqlalchemy as sqla
from sqlalchemy import create_engine

DB_URI = os.getenv('CD_DWH')
engine = create_engine(DB_URI)

In [2]:
# Checking that the Kernel is using the Conda environment datasci-congressional-data
# Below you should see something like '/Users/Username/anaconda3/envs/datasci-congressional-data/bin/python
# If you do NOT see "datasci-congressional-data" this means you are not in the right Python Environment
# Please make sure you have gone through the onboarding docs and/or talk to a project lead.
sys.executable

'/anaconda/envs/congressdata/bin/python'

Below are the tables that currently exist in the database!

In [3]:
QUERY = """
select *
from information_schema.tables
where table_schema not in ('information_schema', 'pg_catalog', 'public')
"""
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)
results.head(100)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,datascicongressionaldata,stg_analytics,stg_candidate_contributions,BASE TABLE,None,None,None,None,None,YES,NO,None
1,datascicongressionaldata,stg_analytics,stg_candidate_election_results,BASE TABLE,None,None,None,None,None,YES,NO,None
2,datascicongressionaldata,data_ingest,sfdata__campaign_finance_form460_schedulea,BASE TABLE,None,None,None,None,None,YES,NO,None
3,datascicongressionaldata,trg_analytics,candidate_contributions,BASE TABLE,None,None,None,None,None,YES,NO,None
4,datascicongressionaldata,data_ingest,maplight__california_candidate,BASE TABLE,None,None,None,None,None,YES,NO,None
5,datascicongressionaldata,data_ingest,maplight__california_other,BASE TABLE,None,None,None,None,None,YES,NO,None
6,datascicongressionaldata,data_ingest,casos__california_candidate_statewide_election...,BASE TABLE,None,None,None,None,None,YES,NO,None
7,datascicongressionaldata,data_ingest,ceda__california_candidate_local_election_results,BASE TABLE,None,None,None,None,None,YES,NO,None


## Query Example

In [ ]:
%%time
QUERY = """
select
  *
from trg_analytics.candidate_contributions
"""
# removed limit 1000 from query (I want to pull all the rows in the table, not just 1000)
with engine.begin() as conn:
    results = pd.read_sql(QUERY, conn)

In [ ]:
results.head(100)

In [ ]:
results.shape

In [ ]:
# List out all the rows in the dataframe
print(list(results.index))
# List out all the columns in the dataframe
print(list(results.columns))
# List out the rows, columns in the dataframe
print(list(results.shape))

In [ ]:
# List out all the unique values in a column
results.recipient_committee_name.unique()
# List out all the unique values by column
for col in results.columns:
    print (col + " : " + str(results[col].unique()))

In [ ]:
# List out all rows using location based indexing
results.iloc[:,2]
# List out all rows using keyword based indexing
results.loc[:, 'election_cycle']

# Filter out rows according to column values
results.loc[(results['donor_state'] == 'CA') & (results['donor_organization'] == 'OTH')]

In [ ]:
results[(results['donor_state'] == 'CA') & (results['donor_organization'] == 'OTH')][['transaction_id','transaction_amount']]

In [ ]:
# Group according to category and summarize
results.groupby('recipient_candidate_name')['transaction_amount'].describe()
# Group according to multiple categories and summarize
results.groupby(['recipient_candidate_name', 'transaction_amount'])['transaction_amount'].count()

In [ ]:
# Find cumulative sum by row
results['transaction_amount'].cumsum()

In [ ]:
results.iloc[2,:]

In [ ]:
len(results.index)

In [ ]:
results.head()

In [ ]:
results['transaction_amount'].sum()

In [ ]:
results.groupby('donor_zip_code').sum().sort_values('transaction_amount', ascending = False)

In [ ]:
topspend = ['95814', '95030', '91506', '94010', '90071']
results[results['donor_zip_code'].isin(topspend)][['donor_city', 'donor_state']]

# results[(results['donor_state'] == 'CA') & (results['donor_organization'] == 'OTH')][['transaction_id','transaction_amount']]
# results.loc[(results['donor_state'] == 'CA') & (results['donor_organization'] == 'OTH')]

In [ ]:
results[(results['election_cycle'] == '2003')].groupby('recipient_candidate_name').sum().sort_values('transaction_amount', ascending = False)

In [ ]:
results[(results['recipient_candidate_name'] == 'POIZNER, STEVE')].groupby('donor_name').sum().sort_values('transaction_amount', ascending = False)

In [ ]:
donorsum = results.groupby('donor_name').sum().sort_values('transaction_amount', ascending = False)
donorsum

In [ ]:
donorsum['cumsum'] = donorsum['transaction_amount'].cumsum()
donorsum['percentile'] = donorsum['cumsum'] / donorsum['transaction_amount'].sum()
donorsum[(donorsum['percentile'] <= .5)].shape

### 1) How many rows are in trg_analytics.candidate_contributions?

In [ ]:
results.shape[0]

### 2) What is the total sum of donations made to candidates in the 2017-2018 election cycle?

In [ ]:
round(results.groupby('election_cycle')['transaction_amount'].sum()['2017'])

### 3) Across history what are the top 5 zip codes ranked according to the donations sourced from those areas? Which cities are these zip codes located in? Do you notice anything surprising?

In [ ]:
topfive_donors_zipcodes = results.groupby('donor_zip_code').sum()['transaction_amount'].nlargest(n=5)
topfive_donors_zipcodes_list = list(topfive_donors_zipcodes.index)
topfive_donors_zipcodes_list

In [ ]:
for zipcode in topfive_donors_zipcodes_list:
    print (results[results['donor_zip_code'] == zipcode]['donor_city'].str.lower().unique())

## 4) In the 2015-2016 election cycle, which candidate received the most donations?

In [ ]:
top_donations = results[(results['election_cycle'] == '2015')].groupby('recipient_candidate_name').sum().sort_values('transaction_amount', ascending = False)
top_donations.iloc[0:5, 1]

### 5) For the candidate you found in question (4) above, who was their highest contributing donor?

In [ ]:
top_donors = results[(results['recipient_candidate_name'] == 'NEWSOM, GAVIN')].groupby('donor_name').sum().sort_values('transaction_amount', ascending = False)
top_donors.iloc[0:5, 1]

### 6) In the 2017-2018 election cycle, how many donors comprised of 50% of all donations made?

In [ ]:
top_donors_2017 = results[(results['election_cycle'] == '2017')].groupby('donor_name').sum().sort_values('transaction_amount', ascending=False)
top_donors_2017['Transaction_Amount_Cumulative_Sum'] = top_donors_2017['transaction_amount'].cumsum()
total_sum = top_donors_2017['transaction_amount'].sum()
half_sum = total_sum / 2

In [ ]:
top_50_donor_count = top_donors_2017[top_donors_2017['Transaction_Amount_Cumulative_Sum'] < half_sum]
top_50_donor_count['Transaction_Amount_Cumulative_Sum'].count()

### 7) Repeat (6) for all the other election cycles (currently the dataset includes election cycles starting in 2001). Does the number of donors who comprise of 50% of all donations made within each election cycle change over time?

In [ ]:
unique_election_cycles = results['election_cycle'].unique()

In [ ]:
dap = {}
for el_cycle in unique_election_cycles:
    top_donors_el_cycle = results[(results['election_cycle'] == el_cycle)].groupby('donor_name').sum().sort_values('transaction_amount', ascending=False)
    top_donors_el_cycle['Transaction_Amount_Cumulative_Sum'] = top_donors_el_cycle['transaction_amount'].cumsum()
    total_sum_cycle = top_donors_2017['transaction_amount'].sum()
    half_sum_cycle = (total_sum_cycle / 2)
    top_50_donor_count_el_cycle = top_donors_el_cycle[top_donors_el_cycle['Transaction_Amount_Cumulative_Sum'] < half_sum]
    dap[el_cycle] = top_50_donor_count_el_cycle.count()
    print (str(el_cycle) + " : " + str(dap[el_cycle]['Transaction_Amount_Cumulative_Sum']))

In [ ]:
top_donors_2005 = results[(results['election_cycle'] == '2005')].groupby('donor_name').sum().sort_values('transaction_amount', ascending=False)
top_donors_2005['Transaction_Amount_Cumulative_Sum'] = top_donors_2005['transaction_amount'].cumsum()
total_sum = top_donors_2005['transaction_amount'].sum()
half_sum = total_sum / 2
# top_50_donor_count = top_donors_2017[top_donors_2017['Transaction_Amount_Cumulative_Sum'] < half_sum]
# top_50_donor_count['Transaction_Amount_Cumulative_Sum'].count()
top_donors_2005

In [ ]:
half_sum